# Variational Principle method

This notebook follows (Thijssen, 2013, p. 29). The variational method is a method for deducing upper bounds of energy eigenstates (usually used on the ground state). 

## Theory 

Consider a subset of Hilbert space whose energy expectation values are greater than some eigenstate's energy that we want to find. It can be shown that the min energy expectation value of any superposition of these states is greater than or equal to the eigenenergy we want (see (Thijssen, 2013, p. 30) for the full proof or (Griffiths, 2018, p. 327) for the simple case).

This can be reformed mathematically into a matrix problem.

If the subset of states is $\{\ket{n}|n=1,2,3,...\}$ (this set doesn't have to be orthonormal), then a general ket takes the form:

$\ket{\phi} = \sum\limits_{i = 1}^{\infty}C_i\ket{i}$

$\implies \bra{\phi}\hat{H}\ket{\phi} = \dfrac{\sum\limits_{i,j = 1}^{\infty}C_i^{*}C_jH_{ij}}{\sum\limits_{i,j = 1}^{\infty}C_i^{*}C_jS_{ij}}$

where $H_{ij} = \bra{i}\hat{H}\ket{j}$ and $S_{ij} = \bra{i}\hat{S}\ket{j}$

Minimising $\bra{\phi}\hat{H}\ket{\phi}$ by calculus gives:

$\sum\limits_{j = 1}^{\infty}C_j(H_{ij} - E S_{ij})$ for all i

This can be rewritten as a matrix equation $\bf{H}\bf{C} = E \bf{S}\bf{C}$

## References

Thijseen J., 2013. Computational Physics. Cambridge: Cambridge University Press

Griffiths D., 2018. Introduction To Quantum Mechanics.  Cambridge: Cambridge University Press